In [1]:
import requests

### 1. API를 이용하여 json포맷으로 데이터를 받기
- string타입으로 받아서 json object로 parsing

##### example 1: 네이버 주식데이터 크롤링

In [5]:
from pandas.io.json import json_normalize

In [6]:
def make_url(pageSize=10, page=1):
    return "http://m.stock.naver.com/api/json/sise/siseListJson.nhn?\
    menu=market_sum&sosok=0&pageSize=" + str(pageSize) + "&page=" + str(page)
make_url()

'http://m.stock.naver.com/api/json/sise/siseListJson.nhn?    menu=market_sum&sosok=0&pageSize=10&page=1'

In [10]:
# 데이터를 가져와서 pandas의 dataframe으로 만드는 함수1
def get_data(url):
    response = requests.get(url) # >>> <Response [200]> 
    json_info = response.json() # >>> 커다란 key, value의 집합
    companys = json_info['result']['itemList'] # 주식정보만 뽑아낸다.
    return json_normalize(companys) # 바로 dataframe으로 만들어줌.

get_data(make_url())

,aa,aq,cd,cr,cv,mks,mt,nm,nv,pcv,rf
0,713265,15174988,005930,0.86,400,3017083,0,삼성전자,47000,46600,2
1,323195,3810122,000660,0.12,100,612250,0,SK하이닉스,84100,84000,2
2,371409,1266890,068270,-5.74,-17500,360513,0,셀트리온,287500,305000,5
3,57115,1541365,005935,-0.54,-200,334343,0,삼성전자우,37000,37200,5
4,103542,298893,005490,-4.49,-16000,296435,0,POSCO,340000,356000,5
5,97988,739540,005380,-2.59,-3500,289664,0,현대차,131500,135000,5
6,93234,239152,207940,-6.50,-26500,252089,0,삼성바이오로직스,381000,407500,5
7,104845,294913,051910,-4.64,-17000,246720,0,LG화학,349500,366500,5
8,56925,1041320,105560,-0.72,-400,230379,0,KB금융,55100,55500,5
9,76132,110287,035420,-0.86,-6000,226783,0,NAVER,688000,694000,5


In [12]:
# 데이터를 가져와서 pandas의 dataframe으로 만드는 함수2
def get_data2(url):
    response = requests.get(url) # >>> <Response [200]> 
    json_info = response.json() # >>> 커다란 key, value의 집합
    companys = json_info['result']['itemList'] # 주식정보만 뽑아낸다.
    
    df = pd.DataFrame(columns=["종목","시세","전일비","등락율","시가총액","거래량"])
    for company in companys:
        df.loc[len(df)] = {
            "종목" : company['nm'],
            "시세" : company['nv'],
            "전일비" : company['cv'],
            "등락율" : company['cr'],
            "시자총액" : company['mks'],
            "거래량" : company['aq'],
        }
    return df

get_data2(make_url())

,종목,시세,전일비,등락율,시가총액,거래량
0,삼성전자,47000,400,0.86,NaN,15174988
1,SK하이닉스,84100,100,0.12,NaN,3810122
2,셀트리온,287500,-17500,-5.74,NaN,1266890
3,삼성전자우,37000,-200,-0.54,NaN,1541365
4,POSCO,340000,-16000,-4.49,NaN,298893
5,현대차,131500,-3500,-2.59,NaN,739540
6,삼성바이오로직스,381000,-26500,-6.50,NaN,239152
7,LG화학,349500,-17000,-4.64,NaN,294913
8,KB금융,55100,-400,-0.72,NaN,1041320
9,NAVER,688000,-6000,-0.86,NaN,110287


##### example 2: dark sky api로 날씨 데이터 크롤링(api json, forecastio 사용)

##### dark sky api
- 위도와 경도를 입력하면 날씨 정보를 보내주는 api

In [14]:
FORECAST_TOKEN = "1da567ab436e650c7af58441bd4bdbae" # 나에게 부여된 dark sky 토큰값

In [18]:
def forecast(lat, lng): # 위도, 경도 
    url = url = "https://api.darksky.net/forecast/{}/{},{}".format(FORECAST_TOKEN, lat, lng)
    response = requests.get(url)
    json_info = response.json()
    return json_info['timezone'], json_info['hourly']['summary']


In [19]:
lat = 37.512
lng = 126.954
forecast(lat, lng)

('Asia/Seoul', 'Mostly cloudy until tomorrow afternoon.')

##### forecastio를 이용하여 날씨데이터 가져오기

In [22]:
def forecast2(lat, lng):
    forecast = forecastio.load_forecast(FORECAST_TOKEN, lat, lng) # 객체 생성
    byHourly = forecast.hourly()
    return byHourly.summary
forecast2(37.512, 126.954)

'Mostly cloudy until tomorrow afternoon.'

## 2. HTML코드를 이용하기
- HTML코드를 string으로 가져와 css셀렉터로 데이터를 select
- selecting한 데이터를 bs4로 parsing

In [27]:
import requests

In [28]:
from bs4 import BeautifulSoup

##### example : 네이버 인기검색어 크롤링

In [29]:
def naver():
    df = pd.DataFrame(columns=["rank","keywords"])
    response = requests.get("https://www.naver.com/")
#     print(response.content)
    dom = BeautifulSoup(response.content, "html.parser")
#     print(dom)
    keywords = dom.select(".ah_roll_area > .ah_l > .ah_item")
#     print(keywords[0].select_one(".ah_r").text)
    for keyword in keywords:
        df.loc[len(df)] = {
            "rank" : keyword.selet_one(".ah_r").text,
            "keywords" : keyword.select_one(".ah_k").text,
        }
        
    return df
        
naver_df = naver()
naver_df["keywords"]

TypeError: 'NoneType' object is not callable